<a href="https://colab.research.google.com/github/buwuyou/CTCN_2020_EE/blob/master/CTCN_time_series_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This tutorial will establish a foundation for time series analysis of remote sensing data (Sentinel-2 time series data), including:

1.

2.

3.


Prerequisites: CTCN 101

### Install, Import, Authenticate and Initialize EE Python API

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [1]:
!pip install -q earthengine-api

# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=sYscag7kjj5rUdJNVzfyTtr1Xghma4qQNq9h2AcVxhI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xgGeQotoeKlEk4gMDyn-J5xId2IgJlSbLlO6xGPiB1y5WWv8PcNCz4E

Successfully saved authorization token.


In [2]:
from pprint import pprint 

import folium
from folium import plugins
print(folium.__version__)
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

0.8.3


# NDVI 101

The normalised difference vegeation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing assessed greenness of vegetation.

>$NDVI=(NIR-R)/(NIR+R)$

In [0]:
def addNDVI(img):
    """A function to compute Sentinel-2 NDVI."""
    ndvi = img.expression('float(b("B8") - b("B4")) / (b("B8") + b("B4"))').rename('ndvi')   
    return img.addBands([ndvi])

In [0]:
def ESAcloudMask(img):
    ''' Sentinel-2 Bits 10 & 11 are clouds & cirrus, so set to 0. '''
    qa = img.select('QA60')
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return img.updateMask(clear) 

In [0]:
# // This function adds a band representing the image timestamp.
def addTime(image):
  # Convert milliseconds from epoch to years to aid in interpretation of the following trend calculation.
  return image.addBands(image.metadata('system:time_start').divide(1000 * 60 * 60 * 24 * 365))


In [0]:
def s2col_monthly(feat):
      bandNamesAll = ['ndvi']
      monthDifference = ee.Date('2019-01-01').advance(1, 'month').millis().subtract(ee.Date('2019-01-01').millis());
      listMap = ee.List.sequence(ee.Date('2016-01-01').millis(), ee.Date('2020-01-01').millis(), monthDifference);

      s2monthIC = (ee.ImageCollection.fromImages(listMap.map(lambda dateMillis: (
         (ee.ImageCollection('COPERNICUS/S2')
                            .filterBounds(feat)
                            .filter(ee.Filter.And(
                              ee.Filter.date(ee.Date(dateMillis), ee.Date(dateMillis).advance(1, 'month')),
                              ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90)))
                            .map(ESAcloudMask)
                            
                            )
         .median()
         .clip(feat)
         .set('system:time_start',dateMillis)
         .set('year', ee.Date(dateMillis).get('year'))
         .set('month', ee.Date(dateMillis).get('month'))
         )
      )))
      
      return s2monthIC.map(addNDVI).select(bandNamesAll).map(addTime)

In [0]:
AOI = ee.Geometry.Polygon(
        [[[
              89.40673828125,
              21.37124437061831
            ],
            [
              90.90087890624999,
              21.37124437061831
            ],
            [
              90.90087890624999,
              22.471954507739227
            ],
            [
              89.40673828125,
              22.471954507739227
            ],
            [
              89.40673828125,
              21.37124437061831
            ]]])

In [54]:
# Fetch a Sentinel-2 monthly images.
imgs = s2col_monthly(AOI)
pprint ({'Sentinel-2 Monthly Images since Jan, 2016': imgs.size().getInfo()})
pprint ({'Sentinel-2 Monthly Image': imgs.first().getInfo()})

{'Sentinel-2 Monthly Images since Jan, 2016': 48}
{'Sentinel-2 Monthly Image': {'bands': [{'crs': 'EPSG:4326',
                                         'crs_transform': [1, 0, 0, 0, 1, 0],
                                         'data_type': {'precision': 'double',
                                                       'type': 'PixelType'},
                                         'dimensions': [2, 2],
                                         'id': 'ndvi',
                                         'origin': [89, 21]},
                                        {'crs': 'EPSG:4326',
                                         'crs_transform': [1, 0, 0, 0, 1, 0],
                                         'data_type': {'precision': 'double',
                                                       'type': 'PixelType'},
                                         'id': 'system:time_start'}],
                              'properties': {'month': 1,
                                             'system:fo

In [40]:
#  Select the bands to model with the independent variable first.
trend = imgs.select(['system:time_start', 'ndvi']).reduce(ee.Reducer.linearFit())# Compute the linear trend over time.
pprint ({'Sentinel-2 Trend Image': trend.getInfo()})

{'Sentinel-2 Trend Image': {'bands': [{'crs': 'EPSG:4326',
                                       'crs_transform': [1, 0, 0, 0, 1, 0],
                                       'data_type': {'precision': 'double',
                                                     'type': 'PixelType'},
                                       'id': 'scale'},
                                      {'crs': 'EPSG:4326',
                                       'crs_transform': [1, 0, 0, 0, 1, 0],
                                       'data_type': {'precision': 'double',
                                                     'type': 'PixelType'},
                                       'id': 'offset'}],
                            'type': 'Image'}}


In [57]:
orange2greenPalette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
mapid = trend.getMapId({'bands': ['scale'], 'min': -0.1, 'max': 0.1, 'palette': orange2greenPalette})
map = folium.Map(location=[22., 90.], zoom_start=10)
# Add custom basemaps
basemaps['Google Maps'].add_to(map)
basemaps['Google Satellite Hybrid'].add_to(map)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Sentinel-2 Trend Map 2016-2019',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [61]:
# change the number to any value between 1988 and 2019, then the according year ESI image will be displayed below, the greener the image is, the high potential of salinized soil concurs
yearIDstart = 2016
yearIDend = 2019
monthID = 2

mapid = imgs.select('ndvi').filter(ee.Filter.eq('year',yearIDstart)).filter(ee.Filter.eq('monthID',monthID)).getMapId({'min': -0.8, 'max': 0.8, 'palette': orange2greenPalette})
mapid1 = imgs.select('ndvi').filter(ee.Filter.eq('year',yearIDend)).filter(ee.Filter.eq('monthID',monthID)).getMapId({'min': -0.8, 'max': 0.8, 'palette': orange2greenPalette})
map = folium.Map(location=[22., 90.], zoom_start=7)
folium.TileLayer(
    tiles=mapid1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Sentinel NDVI for year {yearIDstart} and month {monthID}',
  ).add_to(map)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Sentinel NDVI for year {yearIDend} and month {monthID}',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [0]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt
from pprint import pprint

# Select NDVI, scale them, and sample 500 points.
water = ee.Geometry.Point([
          89.89700317382812,
          22.11999383809455
        ])
samp_fc = img.sample(region=water,scale=10)

# Save server-side ee.List as a client-side Python list.
samp_data = samp_fc.toDictionary().values().getInfo()

# # Display a scatter plot of Red-NIR sample pairs using matplotlib.
# plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
# plt.xlabel('Red', fontsize=12)
# plt.ylabel('NIR', fontsize=12)
# plt.show()

In [0]:
pprint ({'test:':samp_data[0]})

{'test:': ['0_ndvi',
           '1_ndvi',
           '2_ndvi',
           '3_ndvi',
           '4_ndvi',
           '5_ndvi',
           '6_ndvi',
           '7_ndvi',
           '8_ndvi',
           '9_ndvi',
           '10_ndvi',
           '11_ndvi']}


In [0]:
# Arrange the sample as a list of lists.
samp_fc = img.sample(scale=10, numPixels=500)
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(12), ['0_ndvi','1_ndvi','2_ndvi','3_ndvi','4_ndvi','5_ndvi','6_ndvi','7_ndvi','8_ndvi','9_ndvi','10_ndvi','11_ndvi'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

In [0]:
print (len(samp_data[0]))

270
